## Подготовка к предобработке

### Установка и импорт необходимых библиотек

In [30]:
!pip install nltk
nltk.download('punkt')

Defaulting to user installation because normal site-packages is not writeable


[nltk_data] Downloading package punkt to /home/sh1ron/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [31]:
import sqlite3
import nltk
from nltk.tokenize import word_tokenize

### Подключение к базе данных и извлечение постов

In [25]:
conn = sqlite3.connect('/home/sh1ron/HSE/MLOps_pipeline/database.db')
cursor = conn.cursor()

cursor.execute("SELECT * FROM posts")
posts = cursor.fetchall()

conn.close()

🎅 Кто-то в этом году себя ооочень хорошо вёл
